In [ ]:
import cv2
import numpy as np
import pandas as pd
from skimage.morphology import skeletonize
from shapely.geometry import Polygon
from matplotlib import pyplot as plt
from shapely.geometry import Polygon, Point, LineString
from shapely.ops import unary_union

def maxDeviationThresh(hist):
    maximum = max(hist)
    index_max = list(hist).index(maximum)
    index_min = 0
    for i in range(0, index_max):
        if not hist[i] and hist[i + 1]:
            index_min = i
            break

    distances = []
    x1 = index_min
    y1 = hist[index_min]
    x2 = index_max
    y2 = hist[index_max]
    for i in range(index_min + 1, index_max):
        x0 = i
        y0 = hist[i]
        distance = np.abs((y2 - y1) * x0 - (x2 - x1) * y0 + x2 * y1 - y2 * x1) / np.sqrt(
            (y2 - y1) ** 2 + (x2 - x1) ** 2)
        distances.append(distance)
    if index_min < index_max - 1:
        T_index = distances.index(max(distances))
    else:
        T_index = -index_min
    return T_index + index_min


def extract_drawing(image):
    dst = cv2.bilateralFilter(image, 10, sigmaColor=15, sigmaSpace=15)
    # dst = img.copy()
    # max_occ = np.bincount(dst[dst > 0]).argmax()
    # dst[dst == 0] = max_occ
    threshed = np.ones(dst.shape, np.uint8) * 255
    if np.any(dst < 255):
        hist, _ = np.histogram(dst[dst < 255].flatten(), range(257))
        thresh_val = maxDeviationThresh(hist)
        #print(thresh_val)
        mask = dst < thresh_val
        threshed[mask] = 0
    return threshed

def getBackground(external, img, morph=True, ret_hier=False):
    background = np.zeros_like(img)
    points = np.array([external]).reshape((4, 1, 2))
    background = cv2.fillConvexPoly(background, points, (255, 255, 255))
    background = cv2.bitwise_and(img, background)    
    '''overlap = cv2.polylines(cv2.cvtColor(img.copy(), cv2.COLOR_GRAY2RGB), [points], True, (255, 0, 0), 1)
    if internal is not None:
      overlap = cv2.polylines(overlap, [int_points], True, (255, 0, 0), 1)
      plt.imshow(overlap)
      plt.show()'''
    background[background == 0] = 255
    background = extract_drawing(background)
    if morph:
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9, 9))
        # background = cv2.bitwise_not(background)
        background = cv2.bitwise_not(cv2.erode(background, kernel))
        background = skeletonize(background / 255, method='lee').astype(np.uint8)
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
        background = cv2.dilate(background, kernel)
    else:
        background = cv2.bitwise_not(background)
        background = skeletonize(background / 255, method='lee').astype(np.uint8)
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
        background = cv2.dilate(background, kernel)   
    cnts, hier = cv2.findContours(background, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    if ret_hier:
        return background, cnts, hier
    else:
        return background, cnts

def draw_contours(drawing, contour):
  temp = np.zeros_like(drawing)
  red_mask = np.all(drawing == [255, 0, 0], axis=-1)  
  temp[red_mask] = drawing[red_mask]
  drawing = cv2.drawContours(drawing, contour, -1, (0, 255, 0), 2)
  drawing[red_mask] = temp[red_mask]
  return drawing


In [ ]:
h_dist = 25  # distanza diviso 2
v_dist = 132

class Pattern14:
  def __init__(self, img, drawing, r_points):
    self.img = img    
    self.drawing = drawing
    p_dst_r = [(r_points[0] - h_dist, r_points[1]), (r_points[0] + h_dist, r_points[1]),
               (r_points[0] + h_dist, r_points[1] + v_dist), (r_points[0] - h_dist, r_points[1] + v_dist)]
    pad = 50
    self.external = [(p_dst_r[0][0] - pad, p_dst_r[0][1] - pad), (int(p_dst_r[1][0] + 1.5 * pad), p_dst_r[1][1] - pad),
                (int(p_dst_r[2][0] + 1.5 * pad), int(p_dst_r[2][1] + 1.5 * pad)), (p_dst_r[3][0] - pad, int(p_dst_r[3][1] + 1.5 * pad))]

  def tree(self, hier, selected):
    if len(selected) == 1:
        return selected[0]
    idx = selected[0]
    if hier[0][idx][2] != -1:
        return self.tree(hier, selected[1:])
    else:
        return selected[0]

  def get_score(self, r_points, diag1, diag2):
    rhomb = None
    background_r, cnts_r, hier_r = getBackground(self.external, self.img, False, True)
    pixel_rhomb = np.sum(np.divide(background_r, 255))
    ok_idx = []
    wrong_shapes = []
    for c in range(len(cnts_r)):
        self.drawing = draw_contours(self.drawing, [cnts_r[c]])
    for c in range(len(cnts_r)):
        peri = cv2.arcLength(cnts_r[c], True)
        approx = cv2.approxPolyDP(cnts_r[c], 0.05 * peri, True)
        hull = cv2.convexHull(approx)
        confront = [h in approx for h in hull]
        if len(approx) == 4 and peri > 100 and np.all(confront):
          ok_idx.append(c)
            # approx = enlargeRhomb(approx, 1)
            # approx = cv2.approxPolyDP(approx, 0.05 * peri, True)
        elif len(approx) != 4 and peri > 100 and np.all(confront):
          wrong_shapes.append(c)
    if len(ok_idx) > 0:
        selected_c_idx = selected_c_idx = self.tree(hier_r, ok_idx)
        peri = cv2.arcLength(cnts_r[selected_c_idx], True)
        approx = cv2.approxPolyDP(cnts_r[selected_c_idx], 0.05 * peri, True)
        rhomb = np.array([coord[0] for coord in approx])
        rect = cv2.minAreaRect(approx)
        (x, y), (width, height), angle = rect               
        # print('orientation = {}'.format(angle))
        self.drawing = cv2.polylines(self.drawing, [approx], True, (0, 191, 255), 2)
        for vert in approx:
            self.drawing = cv2.circle(self.drawing, tuple(vert[0]), 5, (0, 0, 255), -1)
    if rhomb is not None:
        rhomb_points = []
        sort = np.argsort(rhomb[:, 1])
        for p in sort:
            self.drawing = cv2.circle(self.drawing, tuple(rhomb[p]), 20, (255, 0, 0), 2)
            rhomb_points.append(Point(tuple(rhomb[p])).buffer(20))
        rhomb_points.append(Polygon(rhomb).buffer(1.5),)
        rhomb_fig = unary_union(rhomb_points)
        if diag1 is not None and diag2 is not None:
          diag1_fig = LineString([diag1[0], list(r_points)]).buffer(3)
          diag2_fig = LineString([diag2[0], list(r_points)]).buffer(3)
        else:
          diag1_fig = LineString([[852, 219], list(r_points)]).buffer(3)
          diag2_fig = LineString([[852, 537], list(r_points)]).buffer(3)
        p1 = diag1_fig.intersects(rhomb_fig) and diag2_fig.intersects(rhomb_fig)
        if not p1:
          print('PATTERN14: non tocca diagonali')
          label_rhomb = 2
        else:
          label_rhomb = 3        
        return self.drawing, label_rhomb, (rhomb_fig, rhomb_points)
    elif len(wrong_shapes) > 0:
          print('PATTERN14: forma sbagliata')
          for c in wrong_shapes:
            peri = cv2.arcLength(cnts_r[c], True)
            approx = cv2.approxPolyDP(cnts_r[c], 0.05 * peri, True)
            self.drawing = cv2.polylines(self.drawing, [approx], True, (0, 191, 255), 1)
          label_rhomb = 1
    else:
        print('PATTERN14: forma assente')
        label_rhomb = 0
    return self.drawing, label_rhomb, None